In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import utils
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [21]:
import keras
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten, Conv3D, MaxPooling3D
from keras.regularizers import l2,l1,l1_l2
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, TensorBoard
import h5py

In [3]:
train_data_path = '/home/ec2-user/Capstone_Project/code/left-pca16' # Full path should be given here 
train_mask_path = 'mask_updated/train/'
test_data_path = '/home/ec2-user/Capstone_Project/code/right-pca16' # Full path should be given here 
test_mask_path = 'mask_updated/test/'

In [4]:
# Read data form training and test 

In [7]:
X_train,Y_train, num_bands = utils.load_data(data_path = train_data_path, 
                                             masks_path = train_mask_path, 
                                             crop_size = 33, 
                                             num_classes = 6, 
                                             samples = 100000, 
                                             balance = True)

In [8]:
X_test,Y_test, num_bands = utils.load_data(data_path = test_data_path, 
                                           masks_path = test_mask_path, 
                                           crop_size = 33, 
                                           num_classes = 6, 
                                           samples = 100000, 
                                           balance = True)

In [9]:
# creat generato

In [10]:
print(type(X_train), type(Y_train))
print(X_train.shape, Y_train.shape, num_bands)
print(X_test.shape,  Y_test.shape,  num_bands)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(599996, 33, 33, 16) (599996, 6) 16
(600000, 33, 33, 16) (600000, 6) 16


In [11]:
X_train = np.array([X_train[i] for i in range(X_train.shape[0])])

In [12]:
X_test = np.array([X_test[i] for i in range(X_test.shape[0])])

In [13]:
Y_train = np.array([Y_train[i,:] for i in range(Y_train.shape[0])])

In [14]:
Y_test = np.array([Y_test[i,:] for i in range(Y_test.shape[0])])

# UH Model

In [15]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), strides=(1, 1), input_shape=(33, 33, 16), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), strides=(1, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), strides=(1, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, kernel_regularizer=l2(0.01)))
model.add(Activation('softmax')) 

In [16]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [17]:
# https://machinelearningmastery.com/check-point-deep-learning-models-keras/
filepath="Model_1_4_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs_Model_1_4', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint,tbCallBack]

In [ ]:
model.fit(X_train, Y_train,
          epochs = 20,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test), callbacks = callbacks_list)

Train on 599999 samples, validate on 600000 samples
Epoch 1/20
599999/599999 [==============================] - 226s 377us/step - loss: 1.1537 - acc: 0.9431 - val_loss: 0.7934 - val_acc: 0.8501

Epoch 00001: val_acc improved from -inf to 0.85011, saving model to Model_1_4_best.h5
Epoch 2/20
599999/599999 [==============================] - 289s 481us/step - loss: 0.3404 - acc: 0.9688 - val_loss: 0.8065 - val_acc: 0.8226

Epoch 00002: val_acc did not improve from 0.85011
Epoch 3/20
599999/599999 [==============================] - 292s 487us/step - loss: 0.2220 - acc: 0.9756 - val_loss: 0.6178 - val_acc: 0.8578

Epoch 00003: val_acc improved from 0.85011 to 0.85778, saving model to Model_1_4_best.h5
Epoch 4/20
599999/599999 [==============================] - 281s 468us/step - loss: 0.1729 - acc: 0.9789 - val_loss: 0.6269 - val_acc: 0.8310

Epoch 00004: val_acc did not improve from 0.85778
Epoch 5/20
167296/599999 [=======>......................] - ETA: 2:20 - loss: 0.1534 - acc: 0.9807

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



599999/599999 [==============================] - 288s 480us/step - loss: 0.1475 - acc: 0.9809 - val_loss: 0.5325 - val_acc: 0.8295

Epoch 00005: val_acc did not improve from 0.85778
Epoch 6/20
599999/599999 [==============================] - 287s 479us/step - loss: 0.1314 - acc: 0.9825 - val_loss: 0.4582 - val_acc: 0.8711

Epoch 00006: val_acc improved from 0.85778 to 0.87107, saving model to Model_1_4_best.h5
Epoch 7/20
599999/599999 [==============================] - 292s 487us/step - loss: 0.1206 - acc: 0.9834 - val_loss: 0.7473 - val_acc: 0.8375

Epoch 00007: val_acc did not improve from 0.87107
Epoch 8/20
599999/599999 [==============================] - 291s 486us/step - loss: 0.1123 - acc: 0.9842 - val_loss: 0.5529 - val_acc: 0.8427

Epoch 00008: val_acc did not improve from 0.87107
Epoch 9/20
599999/599999 [==============================] - 292s 486us/step - loss: 0.1058 - acc: 0.9851 - val_loss: 0.8402 - val_acc: 0.8011

Epoch 00009: val_acc did not improve from 0.87107
Epoch 1

In [16]:
# model.save('model_1_4.h5')

In [30]:
# model = load_model('model_1_4.h5')

In [33]:
# # this make aure all the layers are trainable
# for layer in model.layers:
#     print(layer.trainable)

# UH Model - Initialize with TruncatedNormal

In [15]:
tn = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=30)

model = Sequential()
model.add(Conv2D(filters = 32, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 input_shape=(33, 33, 16), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn))
model.add(Activation('softmax')) 

In [17]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [18]:
filepath="Model_1_4_tn_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs_Model_1_4_tn', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint,tbCallBack]

In [19]:
model.fit(X_train, Y_train,
          epochs = 20,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test), callbacks = callbacks_list)

Train on 599995 samples, validate on 600000 samples
Epoch 1/20
599995/599995 [==============================] - 240s 400us/step - loss: 1.0005 - acc: 0.9409 - val_loss: 0.7963 - val_acc: 0.8174

Epoch 00001: val_acc improved from -inf to 0.81741, saving model to Model_1_4_tn_best.h5
Epoch 2/20
599995/599995 [==============================] - 235s 392us/step - loss: 0.2553 - acc: 0.9720 - val_loss: 0.4951 - val_acc: 0.8740

Epoch 00002: val_acc improved from 0.81741 to 0.87400, saving model to Model_1_4_tn_best.h5
Epoch 3/20
599995/599995 [==============================] - 236s 393us/step - loss: 0.1819 - acc: 0.9776 - val_loss: 0.4033 - val_acc: 0.9101

Epoch 00003: val_acc improved from 0.87400 to 0.91013, saving model to Model_1_4_tn_best.h5
Epoch 4/20
599995/599995 [==============================] - 234s 390us/step - loss: 0.1492 - acc: 0.9805 - val_loss: 0.4713 - val_acc: 0.8908

Epoch 00004: val_acc did not improve from 0.91013
Epoch 5/20
599995/599995 [===========================

# UH Model - Initialize with TruncatedNormal with kernel_regularizer=l2(0.1)

In [20]:
tn = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=30)

model = Sequential()
model.add(Conv2D(filters = 32, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 input_shape=(33, 33, 16), 
                 kernel_regularizer=l2(0.1),
                 kernel_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.1),
                 kernel_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.1),
                 kernel_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, 
                kernel_regularizer=l2(0.1),
                kernel_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, 
                kernel_regularizer=l2(0.1),
                kernel_initializer= tn))
model.add(Activation('softmax')) 

In [21]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [22]:
filepath="Model_1_4_tn_reg_0.1_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs_Model_1_4_tn_reg_0.1', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint,tbCallBack]

In [ ]:
model.fit(X_train, Y_train,
          epochs = 10,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test), callbacks = callbacks_list)

Train on 599995 samples, validate on 600000 samples
Epoch 1/10
599995/599995 [==============================] - 271s 452us/step - loss: 0.3520 - acc: 0.9568 - val_loss: 0.7647 - val_acc: 0.7634

Epoch 00001: val_acc improved from -inf to 0.76343, saving model to Model_1_4_tn_reg_0.1_best.h5
Epoch 2/10
599995/599995 [==============================] - 278s 464us/step - loss: 0.2871 - acc: 0.9633 - val_loss: 0.7043 - val_acc: 0.7918

Epoch 00002: val_acc improved from 0.76343 to 0.79178, saving model to Model_1_4_tn_reg_0.1_best.h5
Epoch 3/10
599995/599995 [==============================] - 276s 459us/step - loss: 0.2527 - acc: 0.9666 - val_loss: 0.6918 - val_acc: 0.8365

Epoch 00003: val_acc improved from 0.79178 to 0.83651, saving model to Model_1_4_tn_reg_0.1_best.h5
Epoch 4/10
599995/599995 [==============================] - 275s 458us/step - loss: 0.2294 - acc: 0.9693 - val_loss: 0.5227 - val_acc: 0.8609

Epoch 00004: val_acc improved from 0.83651 to 0.86092, saving model to Model_1_

# UH Model - Initialize with TruncatedNormal for Kernel and Bias 

In [12]:
tn = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=30)

model = Sequential()
model.add(Conv2D(filters = 32, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 input_shape=(33, 33, 16), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(Activation('softmax')) 

In [13]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [14]:
filepath="Model_1_4_tn_bias_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs_Model_1_4_tn_bias', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint,tbCallBack]

In [15]:
model.fit(X_train, Y_train,
          epochs = 15,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test), callbacks = callbacks_list)

Train on 599991 samples, validate on 600000 samples
Epoch 1/15
599991/599991 [==============================] - 266s 444us/step - loss: 1.0021 - acc: 0.9404 - val_loss: 0.4780 - val_acc: 0.9265

Epoch 00001: val_acc improved from -inf to 0.92648, saving model to Model_1_4_tn_bias_best.h5
Epoch 2/15
599991/599991 [==============================] - 257s 428us/step - loss: 0.2546 - acc: 0.9722 - val_loss: 0.6479 - val_acc: 0.8411

Epoch 00002: val_acc did not improve from 0.92648
Epoch 3/15
599991/599991 [==============================] - 256s 427us/step - loss: 0.1804 - acc: 0.9780 - val_loss: 0.5058 - val_acc: 0.8722

Epoch 00003: val_acc did not improve from 0.92648
Epoch 4/15
599991/599991 [==============================] - 258s 430us/step - loss: 0.1483 - acc: 0.9806 - val_loss: 0.4641 - val_acc: 0.8725

Epoch 00004: val_acc did not improve from 0.92648
Epoch 5/15
599991/599991 [==============================] - 260s 433us/step - loss: 0.1303 - acc: 0.9824 - val_loss: 0.4152 - val_ac

# UH Model - Initialize with TruncatedNormal for Kernel and Bias (different seed)

In [13]:
tn = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=100)

model = Sequential()
model.add(Conv2D(filters = 32, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 input_shape=(33, 33, 16), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(Activation('softmax')) 

In [14]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [15]:
filepath="Model_1_4_tn_bias_seed_100_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs_Model_1_4_tn_bias_seed_100', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint,tbCallBack]

In [16]:
model.fit(X_train, Y_train,
          epochs = 10,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test), callbacks = callbacks_list)

Train on 599995 samples, validate on 600000 samples
Epoch 1/10
599995/599995 [==============================] - 285s 475us/step - loss: 0.9940 - acc: 0.9416 - val_loss: 0.6612 - val_acc: 0.8482

Epoch 00001: val_acc improved from -inf to 0.84825, saving model to Model_1_4_tn_bias_seed_100_best.h5
Epoch 2/10
599995/599995 [==============================] - 275s 459us/step - loss: 0.2584 - acc: 0.9724 - val_loss: 0.6676 - val_acc: 0.7723

Epoch 00002: val_acc did not improve from 0.84825
Epoch 3/10
599995/599995 [==============================] - 279s 465us/step - loss: 0.1839 - acc: 0.9776 - val_loss: 0.5805 - val_acc: 0.8290

Epoch 00003: val_acc did not improve from 0.84825
Epoch 4/10
599995/599995 [==============================] - 290s 483us/step - loss: 0.1495 - acc: 0.9802 - val_loss: 0.5716 - val_acc: 0.8570

Epoch 00004: val_acc improved from 0.84825 to 0.85699, saving model to Model_1_4_tn_bias_seed_100_best.h5
Epoch 5/10
599995/599995 [==============================] - 290s 48

## UH Model - Initialize with TruncatedNormal for Kernel and Bias L1-L2 Regularization

In [29]:
tn = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=100)

model = Sequential()
model.add(Conv2D(filters = 32, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 input_shape=(33, 33, 16), 
                 kernel_regularizer=l1_l2(l1=0.01, l2=0.01),
                 kernel_initializer=tn,
                 bias_initializer=tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides= (1, 1), 
                 kernel_regularizer= l1_l2(l1=0.01, l2=0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer= l1_l2(l1=0.01, l2=0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, 
                kernel_regularizer= l1_l2(l1=0.01, l2=0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, 
                kernel_regularizer= l1_l2(l1=0.01, l2=0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(Activation('softmax')) 

In [30]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [31]:
filepath="Model_1_4_l1_l2_reg_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs_Model_1_4_l1_l2_reg', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint,tbCallBack]

In [32]:
model.fit(X_train, Y_train,
          epochs = 15,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test), callbacks = callbacks_list)

Train on 599995 samples, validate on 600000 samples
Epoch 1/15
599995/599995 [==============================] - 301s 502us/step - loss: 5.2209 - acc: 0.8809 - val_loss: 1.3691 - val_acc: 0.7744

Epoch 00001: val_acc improved from -inf to 0.77445, saving model to Model_1_4_l1_l2_reg_best.h5
Epoch 2/15
599995/599995 [==============================] - 269s 449us/step - loss: 0.9547 - acc: 0.9227 - val_loss: 1.1357 - val_acc: 0.8283

Epoch 00002: val_acc improved from 0.77445 to 0.82829, saving model to Model_1_4_l1_l2_reg_best.h5
Epoch 3/15
599995/599995 [==============================] - 273s 456us/step - loss: 0.8920 - acc: 0.9296 - val_loss: 1.2645 - val_acc: 0.7613

Epoch 00003: val_acc did not improve from 0.82829
Epoch 4/15
599995/599995 [==============================] - 275s 458us/step - loss: 0.8652 - acc: 0.9327 - val_loss: 1.3898 - val_acc: 0.6967

Epoch 00004: val_acc did not improve from 0.82829
Epoch 5/15
599995/599995 [==============================] - 301s 502us/step - los

In [ ]:
# UH Model - Initialize with TruncatedNormal for Kernel and Bias L1 Regularization

In [39]:
tn = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=100)

model = Sequential()
model.add(Conv2D(filters = 32, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 input_shape=(33, 33, 16), 
                 kernel_regularizer=l1(0.01),
                 kernel_initializer=tn,
                 bias_initializer=tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides= (1, 1), 
                 kernel_regularizer= l1(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, 
                 kernel_size = (3, 3), 
                 strides=(1, 1), 
                 kernel_regularizer= l1(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, 
                kernel_regularizer= l1(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, 
                kernel_regularizer= l1(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(Activation('softmax')) 

In [40]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [41]:
filepath="Model_1_4_l1_reg_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs_Model_1_4_l1_reg', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint,tbCallBack]

In [42]:
model.fit(X_train, Y_train,
          epochs = 10,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test), callbacks = callbacks_list)

Train on 599995 samples, validate on 600000 samples
Epoch 1/10
599995/599995 [==============================] - 267s 445us/step - loss: 5.0421 - acc: 0.8808 - val_loss: 1.3247 - val_acc: 0.7998

Epoch 00001: val_acc improved from -inf to 0.79976, saving model to Model_1_4_l1_reg_best.h5
Epoch 2/10
599995/599995 [==============================] - 295s 492us/step - loss: 0.9524 - acc: 0.9240 - val_loss: 1.2303 - val_acc: 0.8517

Epoch 00002: val_acc improved from 0.79976 to 0.85170, saving model to Model_1_4_l1_reg_best.h5
Epoch 3/10
599995/599995 [==============================] - 290s 483us/step - loss: 0.8735 - acc: 0.9300 - val_loss: 1.2304 - val_acc: 0.8082

Epoch 00003: val_acc did not improve from 0.85170
Epoch 4/10
599995/599995 [==============================] - 295s 491us/step - loss: 0.8254 - acc: 0.9333 - val_loss: 1.1869 - val_acc: 0.8467

Epoch 00004: val_acc did not improve from 0.85170
Epoch 5/10
599995/599995 [==============================] - 287s 479us/step - loss: 0.8

## UH Model - Initialize with TruncatedNormal for Kernel and Bias (Kernel_Size = (5,5))

In [43]:
tn = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=30)

model = Sequential()
model.add(Conv2D(filters = 32, 
                 kernel_size = (5, 5), 
                 strides=(1, 1), 
                 input_shape=(33, 33, 16), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, 
                 kernel_size = (5, 5), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, 
                 kernel_size = (5, 5), 
                 strides=(1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(Activation('softmax')) 

In [44]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [45]:
filepath="Model_1_4_tn_bias_kernel_size_5_5_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs_Model_1_4_tn_bias_kernel_size_5_5', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint,tbCallBack]

In [46]:
model.fit(X_train, Y_train,
          epochs = 10,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test), callbacks = callbacks_list)

Train on 599995 samples, validate on 600000 samples
Epoch 1/10
599995/599995 [==============================] - 247s 411us/step - loss: 1.1253 - acc: 0.9390 - val_loss: 0.9488 - val_acc: 0.8016

Epoch 00001: val_acc improved from -inf to 0.80156, saving model to Model_1_4_tn_bias_kernel_size_5_5_best.h5
Epoch 2/10
599995/599995 [==============================] - 245s 408us/step - loss: 0.2733 - acc: 0.9692 - val_loss: 0.6221 - val_acc: 0.8441

Epoch 00002: val_acc improved from 0.80156 to 0.84412, saving model to Model_1_4_tn_bias_kernel_size_5_5_best.h5
Epoch 3/10
599995/599995 [==============================] - 253s 422us/step - loss: 0.1901 - acc: 0.9749 - val_loss: 0.7210 - val_acc: 0.8240

Epoch 00003: val_acc did not improve from 0.84412
Epoch 4/10
599995/599995 [==============================] - 247s 412us/step - loss: 0.1578 - acc: 0.9779 - val_loss: 0.8610 - val_acc: 0.7694

Epoch 00004: val_acc did not improve from 0.84412
Epoch 5/10
599995/599995 [===========================

# 3D filter  Conv3D

In [15]:
X_train = X_train[:,:,:,:,np.newaxis]

In [17]:
X_test = X_test[:,:,:,:,np.newaxis]

In [19]:
X_train.shape, X_test.shape

((599996, 33, 33, 16, 1), (600000, 33, 33, 16, 1))

In [22]:
tn = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=30)

model = Sequential()
model.add(Conv3D(filters = 32, 
                 kernel_size = (3, 3, 3), 
                 strides=(1, 1, 1), 
                 input_shape=(33, 33, 16, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling3D(pool_size=(2, 2, 2),strides=(2, 2, 2)))

model.add(Conv3D(filters = 64, 
                 kernel_size = (3, 3, 3), 
                 strides=(1, 1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv3D(filters = 64, 
                 kernel_size = (3, 3, 3), 
                 strides=(1, 1, 1), 
                 kernel_regularizer=l2(0.01),
                 kernel_initializer= tn,
                 bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling3D(pool_size=(2, 2, 2),strides=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(128, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, 
                kernel_regularizer=l2(0.01),
                kernel_initializer= tn,
                bias_initializer= tn))
model.add(Activation('softmax')) 

In [23]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=1e-07, decay=0.0),
              metrics=['accuracy'])

In [24]:
filepath="Model_1_4_3D_kernel_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tbCallBack = TensorBoard(log_dir='./logs_Model_1_4_3D_kernel', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list = [checkpoint,tbCallBack]

In [ ]:
model.fit(X_train, Y_train,
          epochs = 10,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test), callbacks = callbacks_list)

Train on 599996 samples, validate on 600000 samples
Epoch 1/10
599996/599996 [==============================] - 1478s 2ms/step - loss: 1.2278 - acc: 0.9541 - val_loss: 1.3492 - val_acc: 0.6913

Epoch 00001: val_acc improved from -inf to 0.69129, saving model to Model_1_4_3D_kernel_best.h5
Epoch 2/10
599996/599996 [==============================] - 1415s 2ms/step - loss: 0.2289 - acc: 0.9815 - val_loss: 1.4614 - val_acc: 0.6117

Epoch 00002: val_acc did not improve from 0.69129
Epoch 3/10
599996/599996 [==============================] - 1415s 2ms/step - loss: 0.1524 - acc: 0.9850 - val_loss: 0.7107 - val_acc: 0.7915

Epoch 00003: val_acc improved from 0.69129 to 0.79146, saving model to Model_1_4_3D_kernel_best.h5
Epoch 4/10
599996/599996 [==============================] - 1415s 2ms/step - loss: 0.1234 - acc: 0.9866 - val_loss: 1.3573 - val_acc: 0.6611

Epoch 00004: val_acc did not improve from 0.79146
Epoch 5/10
302208/599996 [==============>...............] - ETA: 8:33 - loss: 0.1101 